**가치 평가 클래스**

In [1]:
#
# Dx Library Valuation
# valuation_mcs_american.py
#
import numpy as np

from valuation_class import valuation_class

class valuation_mcs_american(valuation_class):
    '''
    단일 요인 몬테카를로 시뮬레이션 사용한 임의의 페이오프에 대한 아메리칸 옵션 가치 평가 클래스
    
    메서드
    ----
    generate_payoff:
        주어진 경로와 페이오프 함수를 이용하여 페이오프 계산
    present_value:
        롱스태프-슈바르츠 방식에 따른 현재 가치 반환
    '''
    
    def generate_payoff(self,fixed_seed = False):
        try:
            strike = self.strike
        except:
            pass
        paths = self.underlying.get_instrument_values(fixed_seed=fixed_seed)
        time_grid = self.underlying.time_grid
        try:
            time_index_start = int(np.where(time_grid == self.pricing_date)[0])
            time_index_end = int(np.where(time_grid == self.maturity)[0])
        except:
            print("Maturity date not in time grid of underlying.")
        instrument_values = paths[time_index_start:time_index_end + 1]
        try:
            payoff = eval(self.payoff_func)
            return instrument_values, payoff, time_index_start, time_index_end
        except :
            print('Error evaluating payoff function')
    
    def present_value(self,accuracy = 6, fixed_seed = False, bf = 5, full = False):
        instrument_values, inner_values, time_index_start, time_index_end = self.generate_payoff(fixed_seed=fixed_seed)
        time_list = self.underlying.time_grid[time_index_start:time_index_end+1]
        discount_factors = self.discount_curve.get_discount_factors(time_list,dtobjects = True)
        V = inner_values[-1]
        for t in range(len(time_list)-2,0,-1):
            #주어진 구간 할인율 계산
            df = discount_factors[t,1]/discount_factors[t+1,1]
            #회귀 분석 단계
            rg = np.polyfit(instrument_values[t], V * df, bf)
            #경로에 대한 보유 가치 계산
            C = np.polyval(rg,instrument_values[t])
            #최적 결정 단계
            #조건 만족시 내재가치, 아니면 보유가치
            V = np.where(inner_values[t]>C,inner_values[t],V*df)
        df = discount_factors[0,1]/discount_factors[1,1]
        result = df*np.sum(V)/len(V)
        if full:
            return round(result,accuracy),df*V
        else:
            return round(result,accuracy)

**ex**

In [2]:
from dx import *

In [3]:
me_gbm = market_environment('me_gbm', dt.datetime(2015, 1, 1))

In [4]:
me_gbm.add_constant('initial_value', 36.)
me_gbm.add_constant('volatility', 0.2)
me_gbm.add_constant('final_date', dt.datetime(2016, 12, 31))
me_gbm.add_constant('currency', 'EUR')
me_gbm.add_constant('frequency', 'W')
# weekly frequency
me_gbm.add_constant('paths', 50000)

In [5]:
csr = constant_short_rate('csr', 0.06)

In [6]:
me_gbm.add_curve('discount_curve',csr)

In [7]:
gbm = geometric_brownian_motion('gbm',me_gbm)

In [8]:
payoff_func = 'np.maximum(strike-instrument_values,0)'

In [9]:
me_am_put = market_environment('me_am_put', dt.datetime(2015, 1, 1))

In [10]:
me_am_put.add_constant('maturity', dt.datetime(2015, 12, 31))
me_am_put.add_constant('strike', 40.)
me_am_put.add_constant('currency', 'EUR')

In [11]:
am_put = valuation_mcs_american('am_put',underlying=gbm,
                               mar_env=me_am_put,payoff_func=payoff_func)

In [12]:
%time am_put.present_value(fixed_seed=True,bf = 5)

Wall time: 3.33 s


4.64108

In [13]:
%%time
ls_table = []
for initial_value in(36.,38.,40.,42.,44.):
    for volatility in(0.2,0.4):
        for maturity in (dt.datetime(2015,12,31),
                        dt.datetime(2016,12,31)):
            am_put.update(initial_value = initial_value,
                         volatility=volatility,maturity=maturity)
            ls_table.append([initial_value,volatility,maturity,am_put.present_value(bf=5)])

Wall time: 52.4 s


In [16]:
print('s0|Vola|T|Value')
print(22*'-')
for r in ls_table:
    print('%d|%3.1f|%d|%5.3f'%\
         (r[0],r[1],r[2].year-2014,r[3]))

s0|Vola|T|Value
----------------------
36|0.2|1|4.624
36|0.2|2|5.165
36|0.4|1|7.502
36|0.4|2|9.589
38|0.2|1|3.383
38|0.2|2|4.060
38|0.4|1|6.511
38|0.4|2|8.643
40|0.2|1|2.429
40|0.2|2|3.149
40|0.4|1|5.630
40|0.4|2|7.816
42|0.2|1|1.675
42|0.2|2|2.402
42|0.4|1|4.825
42|0.4|2|7.054
44|0.2|1|1.146
44|0.2|2|1.873
44|0.4|1|4.171
44|0.4|2|6.430


In [14]:
am_put.update(initial_value=36.)
am_put.delta()

-0.4831

In [15]:
am_put.vega()

22.6506